In [1]:
%pip install openai pydantic


   ---------------------------------------- 0.0/730.3 kB ? eta -:--:--
   ---------------------------------------- 730.3/730.3 kB 5.0 MB/s eta 0:00:00

   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   ---------------------------------------- 3/3 [openai]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# !pip install openai pydantic
import openai
from pydantic import BaseModel, ValidationError
from typing import List
import json
import os

# Set your API key (replace with os.getenv("OPENAI_API_KEY") for env variable usage)
openai.api_key = "YOUR_OPENAI_API_KEY"

# Define the expected output format using Pydantic
class MeetingNotes(BaseModel):
    summary: str
    action_items: List[str]

# Function to query OpenAI with retry logic
def extract_meeting_notes(transcript: str) -> dict:
    prompt_template = (
        """You are a meeting assistant.\n"
        "1. Summarize the meeting transcript below in exactly two sentences.\n"
        "2. Then list all action items mentioned, each as a separate bullet beginning with a dash.\n"
        "Return the result strictly as JSON with keys \"summary\" and \"action_items\".\n\n"
        "Transcript:\n{transcript}"""
    )

    def call_gpt(prompt):
        response = openai.ChatCompletion.create(
            model="gpt-4o",  # or gpt-4.1 or gpt-4o-mini
            temperature=0,
            messages=[
                {"role": "system", "content": prompt}
            ]
        )
        return response.choices[0].message.content

    full_prompt = prompt_template.format(transcript=transcript)
    output = call_gpt(full_prompt)

    try:
        parsed = json.loads(output)
        return MeetingNotes(**parsed).dict()
    except (json.JSONDecodeError, ValidationError):
        retry_prompt = (
            "Please output valid JSON only.\n" + full_prompt
        )
        output_retry = call_gpt(retry_prompt)
        parsed_retry = json.loads(output_retry)
        return MeetingNotes(**parsed_retry).dict()

# -----------------------------
# 🔍 Sample Test Case
# -----------------------------
sample_transcript = """
Alice: Welcome everyone. Today we need to finalize the Q3 roadmap.
Bob: I’ve emailed the updated feature list—please review by Friday.
Carol: I’ll set up the user‐testing sessions next week.
Dan: Let’s push the new UI mockups to staging on Wednesday.
Alice: Great. Also, can someone compile the stakeholder feedback into a slide deck?
Bob: I can handle the slide deck by Monday.
Alice: Thanks, team. Meeting adjourned.
"""

# Run the extractor
output = extract_meeting_notes(sample_transcript)
print(json.dumps(output, indent=2))
